# Preprocessing

In [5]:
import librosa
import numpy as np

def preprocess_audio(audio_path, sample_rate=16000):
    """
    Vorverarbeitung einer Audiodatei: Resampling und Normalisierung.

    Args:
        audio_path (str): Pfad zur Audiodatei.
        sample_rate (int): Ziel-Sampling-Rate (Standard: 16000).

    Returns:
        np.array: Das vorverarbeitete Audio-Signal.
    """
    # Audiodatei laden und resamplen
    audio_signal, _ = librosa.load(audio_path, sr=sample_rate)

    # Zu Mono konvertieren (falls mehrkanalig)
    audio_signal = librosa.to_mono(audio_signal)

    # Normalisieren
    max_val = np.max(np.abs(audio_signal))
    if max_val > 0:
        audio_signal = audio_signal / max_val

    return audio_signal

# Funktion reverberation

In [8]:
import numpy as np
import librosa

def calculate_rt60(audio_signal, sample_rate=16000):
    """
    Schätzt die Halligkeit (RT60) eines Audiosignals basierend auf der Schroeder-Rückwärtsintegration.

    Quelle:
        Schroeder, M. R. (1965). 
        "New Method of Measuring Reverberation Time." 
        *The Journal of the Acoustical Society of America*.

    Args:
        audio_signal (np.array): Das normalisierte Audio-Signal (1D-Array).
        sample_rate (int): Sampling-Rate des Signals (Standard: 16000 Hz).

    Returns:
        float: RT60-Wert in Sekunden (oder NaN bei fehlender Schätzung).
    """
    # Berechne das Energieprofil (Squared Signal)
    energy = audio_signal ** 2

    # Falls das Signal zu leise ist, brechen wir ab
    if np.max(energy) == 0:
        return np.nan

    # Rückwärtsintegration (Schroeder-Integration)
    energy_reversed = np.cumsum(energy[::-1])[::-1]

    # Normalisiere die Energie (max = 0 dB)
    energy_db = 10 * np.log10(energy_reversed / np.max(energy_reversed) + 1e-10)  # Schutz vor log(0)

    # Finde -5 dB und -35 dB Punkte (anstatt -60 dB für stabilere Schätzung)
    idx_5dB = np.where(energy_db <= -5)[0]
    idx_35dB = np.where(energy_db <= -35)[0]

    # Falls keine Punkte gefunden wurden, RT60 als NaN zurückgeben
    if len(idx_5dB) == 0 or len(idx_35dB) == 0:
        return np.nan

    # Berechne die Zeitdifferenz
    t_5dB = idx_5dB[0] / sample_rate
    t_35dB = idx_35dB[0] / sample_rate

    # Lineare Regression zur RT60-Schätzung
    rt60 = (t_35dB - t_5dB) * (60 / 30)  # Hochrechnung von -30 dB auf -60 dB

    return rt60

# Test

In [9]:
import os

# RT60 für alle Dateien berechnen
for file_name in sorted(os.listdir("../audio_files")):  # Alphabetische Sortierung
    if file_name.endswith(".wav"):
        file_path = os.path.join("../audio_files", file_name)

        # Audiodatei vorverarbeiten
        audio_signal = preprocess_audio(file_path)

        # RT60 berechnen
        rt60_value = calculate_rt60(audio_signal, sample_rate=16000)

        print(f"RT60 ({file_name}): {rt60_value:.3f} s")

RT60 (_noise_pink.wav): 0.633 s
RT60 (_noise_white.wav): 0.641 s
RT60 (_signal_constant.wav): 0.632 s
RT60 (_signal_silence.wav): nan s
RT60 (_signal_sine.wav): 0.632 s
RT60 (_snr_03.wav): 2.426 s
RT60 (_snr_10.wav): 2.536 s
RT60 (_snr_20.wav): 2.427 s
RT60 (example1.wav): 5.654 s
RT60 (example2.wav): 4.570 s
RT60 (example3.wav): 3.968 s
